# Resumo do código

>### Entra: 1_Ninjas e 2_Delta  
>### Sai: OUT_2_DataFusion_2022
---
O objectivo é receber dados dos Ninjas e da Delta e devolver um ficheiro Wide que permita verificar o comportamento de vários produtos em diversas lojas através de representações gráficas. 

Devolve um ficheiro que entra no R para gerar o Dashboard.

---
- Inputs

> __Dados dos ninjas__ em formato Wide
> - 1's e 0's consoante a presença e ausência do produto, Data e Loja

> __Dados da Delta__ em formato Wide
> - Stocks e trânsito, Sellout dia anterior

- Outputs

> __Ficheiro Wide__ 

> __Métricas novas:__
> - Roturas de Stock, Pré-rotura
> - Sinal
> - Ciclos e Adequação de Stock
> - STK
> - (Novo) Balanço
> - (Novo) Dias para a rotura de stock e de prateleira


In [1]:
import pandas as pd
import numpy as np

def escrever_excel(dfa, nome):
    dfa.to_excel('%s.xlsx' %nome, index=False)

- # `Ninjas`

In [2]:
#Info Ninja
dfNinjas=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\1Ninjas_Limpo.xlsx")

> Definir variáveis importantes
> - Se houver um ficheiro com os produtos

In [3]:
# Ler ficheiro para dataframe
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Padrão_05_2023\\produtos.txt', header=None)

# Passar para uma lista
produtos = df_produtos[0].tolist()
resto = dfNinjas.columns.difference(produtos)

> - Se se quiser usar o primeiro e o último produtos

In [4]:
# Nomes de entrada
#x= input("Primeiro produto: ")                # CAFÉ DELTA Q MYTHIQ 80CAP
#y= input("Último produto: ")                  # BEBIDA CEREAIS DELTA C/20%CAFE FR 200G

# Passar para uma lista
#produtos = dfNinjas.columns[dfNinjas.columns.get_loc(x): dfNinjas.columns.get_loc(y)+1].tolist()
#resto = dfNinjas.columns.difference(produtos)

> - Número de produtos presentes

In [5]:
# Criar coluna com o número dos produtos em causa presentes no instante

dfNinjas["Num_Produtos"] = dfNinjas[produtos].sum(axis=1)

dfNinjas.loc[(dfNinjas["Num_Produtos"] == 0) & (dfNinjas[produtos[1]].isna()), "Num_Produtos"] = np.nan

> - Reposição

In [52]:
# Ler ficheiro
dfRepos=pd.read_excel("D:\\B&N Dados\\Delta\\Reposição_Sonae_Código.xlsx")

# Criar coluna de reposição
dfNinjas['Reposição'] = [1 if val in dfRepos['STORE'].values else 0 for val in dfNinjas['STORE']]

> - Id

In [ ]:
# Extract the day and month components from the 'datetime' column
dfNinjas['day'] = pd.to_datetime(dfNinjas['DATA']).dt.day
dfNinjas['month'] = pd.to_datetime(dfNinjas['DATA']).dt.month

# Create a new column combining day and month
dfNinjas['day_month'] = dfNinjas['day'].astype(str) + '-' + dfNinjas['month'].astype(str)

# Assign unique IDs to each unique day-month combination
dfNinjas['id'], _ = pd.factorize(dfNinjas['day_month'])

dfNinjas = dfNinjas.drop(columns=['day', 'month', 'day_month'])

# <font color=red>Gráficos!!</font>

In [57]:
#escrever_excel(dfNinjas, "IN_5_Gráficos")

# <font color=red>Fim</font>

> Vendedor

In [33]:
# Ler ficheiro
dfVendedor=pd.read_excel("D:\B&N Dados\Delta\Padrão_05_2023\\Vendedor.xlsx", sheet_name = "Lista Lojas Sonae")
dfVendedor = dfVendedor.rename(columns={"Cód. Loja":"STORE"})

# Criar coluna de reposição
dfNinjas = pd.merge(dfNinjas, dfVendedor[["STORE","Vendedor"]], how="left", on = "STORE")

- #  `Delta (stocks, reposição e sellouts)`

In [34]:
# Ler o ficheiro long com Stocks e Fornecimento

dfDelta=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\2Delta_Limpo.xlsx", sheet_name="Sheet1")

# <font color=green>Mergir</font>

In [35]:
# Dataframe mergido no fim
dfMergir=dfNinjas.copy()


for coluna in produtos:       
    
    #Cada dfStocks é um dataframe com o produto específico
    dfStocks=dfDelta[dfDelta["DESC_ARTIGO"]==coluna][["DATA", "STORE", "STOCK", "PRES_STOCK", "INTRANSIT", "EXPECTED", "SELLOUT"]]   #Dados Ninjas, seleccionar colunas do café específico
    
    #Juntar dados de acordo com o dia e a loja
    dfMergir=pd.merge(dfMergir, dfStocks, how="left", on = ["DATA","STORE"])  
    
    #Mudar nomes para ser adaptado a cada produto
    dfMergir=dfMergir.rename(columns={"STOCK": "STOCK %s" % coluna, "PRES_STOCK":"PRESLINEAR %s" % coluna, "INTRANSIT":"INTRANSIT %s" % coluna, "EXPECTED":"EXPECTED %s" % coluna, "SELLOUT":"SELLOUT %s" % coluna})                       #Nomear coluna nova
    
    

In [54]:
dfFinal=dfMergir.copy()

### <font color=green>Fim</font>

# Já temos DataFrame com:
- Informação Ninjas
- Informação Stocks, Linear, Intransit, Expected e Sellout


## Fazer:

- STK 
- Balanço
- Rotura 
- Pré-rotura
- Sinal
- Ciclos
- Adequação de Stock
- Dias para a rotura de stock
- Dias para a rotura de prateleira

> STK

In [37]:
for i in produtos:

    transito = "INTRANSIT %s" % i
    esperado = "EXPECTED %s" % i
    stock  = "STOCK %s" % i
    
    stk = "STK %s" %i
      
    dfFinal[stk] = dfFinal[stock] + dfFinal[esperado] + dfFinal[transito]

> Balanço

In [38]:
for i in produtos:
    stk = "STK %s" %i
    #msa = "MSA %s" %i
    sellout = "SELLOUT %s" %i
    
    balance = "BALANCE %s" %i
    
    dfFinal[balance] = dfFinal[sellout] / dfFinal[stk]
    
    

> Rotura

In [39]:
for i in produtos:
    rotura  = "ROTURA %s" % i
    stock  = "STOCK %s" % i
    preslinear = "PRESLINEAR %s" % i
    
    dfFinal[rotura] = np.where((dfFinal[stock] > 0) & ~(dfFinal[i].isna()), 0, 
                      np.where((dfFinal[stock] <= 0) & (dfFinal[preslinear] > 0) & ~(dfFinal[i].isna()), 1, 
                      np.nan))


> Pré-Rotura

In [40]:
for i in produtos:
    stock  = "STOCK %s" % i
    preslinear = "PRESLINEAR %s" % i
    pre_rotura= "PRÉ-ROTURA %s" % i
    
    dfFinal[pre_rotura] = (dfFinal[stock] < dfFinal[preslinear]).astype(int)

>Sinal

In [41]:
#dfFinal["Sinal"]= pd.Series(dtype='int')

# Define the mappings of numbers to strings
mapping = {1: "Verdadeiramente Correcto",
           2: "Falso Positivo",
           3: "Falso Negativo",
           4: "Rejeição Correcta"}

for i in produtos:
    sinal = "SINAL %s" % i
    stock  = "STOCK %s" % i
    
    dfFinal[sinal] = np.where((dfFinal[stock] > 0) & (dfFinal[i] == 1), 1,
                     np.where((dfFinal[stock] > 0) & (dfFinal[i] == 0), 2,
                     np.where((dfFinal[stock] <= 0) & (dfFinal[i] == 1), 3,
                     np.where((dfFinal[stock] <= 0) & (dfFinal[i] == 0), 4, 
                     np.nan))))


    # Map the numbers to the corresponding strings
    dfFinal[sinal] = dfFinal[sinal].map(mapping)
    

# 1: Há Stock e Sinal Positivo Ninja "Verdadeiramente Correcto"
# 2: Há Stock e Sinal Negativo Ninja "Falso Positivo"
# 3: Não há Stock e Sinal Positivo Ninja "Falso Negativo"
# 4: Não há Stock e Sinal Negativo Ninja "Rejeição Correcta"

> Ciclos

In [42]:
for i in produtos:
    ciclos = "CICLO %s" % i
    stock  = "STOCK %s" % i
    linear = "PRESLINEAR %s" % i
    dfFinal[ciclos]=dfFinal[stock]/dfFinal[linear]

> Adequação

In [43]:
# Define the mappings of numbers to strings
mapping = {1: "Stock Suficiente",
           2: "Stock Insuf c Forn Adequado",
           3: "Stock Insuf c Forn Desadequado"}

for i in produtos:
    adequa = "ADEQUAÇÃO %s" % i
    ciclos = "CICLO %s" % i
    transito = "INTRANSIT %s" % i
    esperado = "EXPECTED %s" % i
    stock  = "STOCK %s" % i
    linear = "PRESLINEAR %s" % i
    
    dfFinal[adequa] = np.where(dfFinal[ciclos] > 1.1, 1,
                      np.where((dfFinal[ciclos] <= 1.1) & (dfFinal[transito] + dfFinal[esperado] + dfFinal[stock] >= dfFinal[linear]), 2,
                      np.where((dfFinal[ciclos] <= 1.1) & (dfFinal[transito] + dfFinal[esperado] + dfFinal[stock] < dfFinal[linear]), 3, 
                      np.nan)))
    
    # Map the numbers to the corresponding strings
    dfFinal[adequa] = dfFinal[adequa].map(mapping)

- Dias para a rotura

In [44]:
for i in produtos:
    stock  = "STOCK %s" % i
    #msa = "MSA %s" %i
    sellout = "SELLOUT %s" % i
    dias_rotura = "Dias_para_Rotura %s" % i
    
    dfFinal[dias_rotura] = dfFinal[stock]/dfFinal[sellout]

- Dias para a rotura de prateleira

In [45]:
dfFinal2=dfFinal.copy()

for i in produtos:
    linear = "PRESLINEAR %s" % i
    #msa = "MSA %s" %i
    sellout = "SELLOUT %s" % i
    dias_rotura_prat = "Dias_para_Rotura_Linear %s" % i
    
    dfFinal2[dias_rotura_prat] = dfFinal2[linear] / dfFinal2[sellout]

# Acabamos com:
- Informação Ninjas
- Informação Stocks, Linear, Intransit e Expected
- Informação Sellouts, Vendedor, Reposição

- STK 
- Balanço
- Rotura
- Pré-Rotura
- Sinal 
- Ciclos
- Adequação de Stock
- Dias para a rotura de stock
- Dias para a rotura de prateleira
- Balanço

# Escrever

In [46]:
escrever_excel(dfFinal2, "OUT_2_DataFusion")
#dfFinalCorr.to_csv('ficheiro_NSS.csv', index=False)

---

# 2022

## Ler ficheiro

In [ ]:
# Ler o ficheiro long com Stocks e Fornecimento
dfDelta_2022=pd.read_excel("D:\\B&N Dados\\Delta\\cPadrão\\2Delta2022Id.xlsx", sheet_name="Sheet1")

## Mergir

In [ ]:
# Dataframe mergido no fim
df_2022=dfNinjas.copy()


for coluna in produtos:       
    
    #Cada dfStocks é um dataframe com o produto específico
    dfStocks=dfDelta_2022[dfDelta_2022["DESC_ARTIGO"]==coluna][["id", "Semana", "STORE", "SOH", "PRES_STOCK", "INTRANSIT", "EXPECTED", "Dia_Actual"]]   #Dados Ninjas, seleccionar colunas do café específico
    
    #Juntar dados de acordo com o dia e a loja
    df_2022=pd.merge(df_2022, dfStocks, how="left", on = ["id","STORE", "Semana"])  
    
    #Mudar nomes para ser adaptado a cada produto
    df_2022=df_2022.rename(columns={"SOH": "STOCK %s" % coluna, "PRES_STOCK":"PRESLINEAR %s" % coluna, "INTRANSIT":"INTRANSIT %s" % coluna, "EXPECTED":"EXPECTED %s" % coluna, "Dia_Actual":"SELLOUT %s" % coluna})                       #Nomear coluna nova
    

## Escrever

In [ ]:
escrever_excel(df_2022, "OUT2DataFusion2022")